In [1]:
from datetime import datetime
import pytz

In [2]:
def get_current_time(timezone: str = 'Asia/Seoul'):
    tz = pytz.timezone(timezone) # 타임존 설정
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S") # 설정한 타임존의 현재 시간을 다음 시간 포맷과 같이 반환 ex) 2025-10-27 20:00:00
    now_timezone = f'{now} {timezone}'
    print(now_timezone)
    return now_timezone

In [3]:
get_current_time('America/New_York')

2025-10-27 22:03:32 America/New_York


'2025-10-27 22:03:32 America/New_York'

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "해당 타임존의 날짜와 시간을 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'timezone': {
                        'type': 'string',
                        'description': '현재 날짜와 시간을 반환할 타임존을 입력하세요. (예: Asia/Seoul)',
                    },
                },
                "required": ['timezone'],
            },
        }
    },
]

In [5]:
if __name__ == '__main__':
    get_current_time('America/New_York')

2025-10-27 22:03:36 America/New_York


In [6]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

In [7]:
load_dotenv()
api_key = os.getenv("OPEN_API_KEY")  # 환경 변수에서 API 키 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

In [8]:
def get_ai_response(messages, tools=None):
    response = client.chat.completions.create(
        model="gpt-4o",  # 응답 생성에 사용할 모델 지정
        messages=messages,  # 대화 기록을 입력으로 전달
        tools=tools,  # 사용 가능한 도구 목록 전달
    )
    return response  # 생성된 응답 내용 반환

In [ ]:
messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},  # 초기 시스템 메시지
]

while True:
    user_input = input("사용자\t: ")  # 사용자 입력 받기

    if user_input == "exit":  # 사용자가 대화를 종료하려는지 확인
        break

    messages.append({"role": "user", "content": user_input})  # 사용자 메시지 대화 기록에 추가

    ai_response = get_ai_response(messages, tools=tools)
    ai_message = ai_response.choices[0].message
    print(ai_message)  # gpt에서 반환되는 값을 파악하기 위해 임시로 추가

    tool_calls = ai_message.tool_calls  # AI 응답에 포함된 tool_calls를 가져옴

    if tool_calls:  # tool_calls가 있는 경우
        for tool_call in tool_calls:
            tool_name = tool_call.function.name # 실행해야한다고 판단한 함수명 받기
            tool_call_id = tool_call.id         # tool_call 아이디 받기
            arguments = json.loads(tool_call.function.arguments) # 문자열을 딕셔너리로 변환

            if tool_name == "get_current_time":  # 만약 tool_name이 "get_current_time"이라면
                messages.append({
                    "role": "function",  # role을 "function"으로 설정
                    "tool_call_id": tool_call_id,
                    "name": tool_name,
                    "content": get_current_time(timezone=arguments['timezone']),  # 타임존 추가
                })

        messages.append({"role": "system", "content": "이제 주어진 결과를 바탕으로 답변할 차례다."})  # 함수 실행 완료 메시지 추가
        ai_response = get_ai_response(messages, tools=tools) # 다시 GPT 응답 받기
        ai_message = ai_response.choices[0].message

    messages.append(ai_message)  # AI 응답을 대화 기록에 추가하기

    print("AI\t: " + ai_message.content)  # AI 응답 출력